In [266]:
import pandas as pd
import pandasql
import csv
import datetime

In [262]:
weather_data = pd.read_csv('C:\\Users\\PC\\Desktop\\Work\\C1\\weather_underground.csv')
q="""
SELECT fog, MAX(maxtempi) FROM weather_data GROUP BY(fog)
"""
foggy_days = pandasql.sqldf(q.lower(), locals())
print(foggy_days)

   fog  max(maxtempi)
0    0             86
1    1             81


In [263]:
#Look at the day of week (optional)
#weather_data['date'] = pd.to_datetime(weather_data['date']).dt.dayofweek

In [264]:
weather_data

,date,maxpressurem,maxdewptm,maxpressurei,maxdewpti,since1julheatingdegreedaysnormal,heatingdegreedaysnormal,since1sepcoolingdegreedaysnormal,hail,since1julsnowfallm,...,precipi,snowfalli,since1jancoolingdegreedaysnormal,precipm,snowfallm,thunder,monthtodateheatingdegreedays,meantempi,maxvism,meantempm
0,2011-05-01,1026,6,30.31,42,4646,8,NaN,0,157.23,...,0.00,0.0,13,0.00,0.0,0,5,60,16,16
1,2011-05-02,1026,10,30.31,50,4653,7,NaN,0,157.23,...,0.00,0.0,14,0.00,0.0,0,13,57,16,14
2,2011-05-03,1021,15,30.14,59,4660,7,NaN,0,157.23,...,0.00,0.0,15,0.00,0.0,0,13,65,16,18
3,2011-05-04,1017,14,30.03,57,4667,7,NaN,0,157.23,...,0.50,0.0,16,12.70,0.0,0,23,55,16,13
4,2011-05-05,1016,4,30.01,39,4673,6,NaN,0,157.23,...,0.00,0.0,17,0.00,0.0,0,32,56,16,13
5,2011-05-06,1015,8,29.99,46,4679,6,NaN,0,157.23,...,0.00,0.0,18,0.00,0.0,0,36,61,16,16
6,2011-05-07,1011,8,29.87,47,4685,6,NaN,0,157.23,...,0.00,0.0,19,0.00,0.0,0,37,64,16,18
7,2011-05-08,1014,9,29.96,48,4691,6,NaN,0,157.23,...,0.00,0.0,20,0.00,0.0,0,39,63,16,17
8,2011-05-09,1016,9,30.01,49,4696,5,NaN,0,157.23,...,0.00,0.0,21,0.00,0.0,0,41,63,16,17
9,2011-05-10,1017,8,30.05,47,4701,5,NaN,0,157.23,...,0.00,0.0,22,0.00,0.0,0,42,64,16,18


In [ ]:
"""
Программа 
1. Takes list of N names of the files (.csv), consisting data from subway turnstiles, give N updated files: update_filenames[i] (in .csv format)
In updated file there is only one data point for one row. (Data before update had several data point in one row).
2. Consolidating updated turnstile files in file master_turnstile.csv
3. Takes "REGULAR" information.
4. Adds 2 columns: difference between entries/exits in this and previous 4 hours (Only between the same source of data).
If there is difference between two sources of data or NaN about exits and entries, puts 0 in ENTRIESn/EXITSn column.
5. TIMEn rows differ only by hours. It's reasonable to represent TIMEn column in hours to make data easier to analyse.
6. Date in weather file presented in YYYY-MM-DD format. To work with both we will format subway from DD-MM-YY to YYYY-MM-DD.
"""

In [332]:
def txt_maker(filenames): #list of files names (with pathes)
    new_files=[]
    for i in range(len(filenames)):
        with open(filenames[i], newline='') as csvfile:
            text = csv.reader(csvfile, delimiter=' ')
            new_rows=[]
        
            for row in text:
                row[0] = row[0].split(',')
                begin=row[0][0:3]
                new_rows.append(row[0][:8])
            
                for i in range((len(row[0])-8)//5):
                    new_rows.append(begin+row[0][8+i*5:8+(i+1)*5])
            new_files.append(new_rows)
    return new_files

def strr(l):
    for i in range(len(l)):
        l[i]=str(l[i])
    return l

def summ(l):
    s=''
    for i in range(len(l)):
        s+=l[i]+','
    s=s[:-1]
    return s

def stringer(l):
    for i in range(len(l)):
        l[i] = summ(strr(l[i]))
    return l

def path(filenames:list, path:str):
    for i in range(len(filenames)):
        filenames[i] = path+filenames[i]
    return filenames

In [333]:
#saving updated data in format updated_filenames[i]

pathh='C:\\Users\\PC\\Desktop\\Work\\C1\\'
filenames = path(['turnstile_110507.csv', 'turnstile_110528.csv'], pathh)

for i in range(len(filenames)):
    with open(filenames[i][:len(pathh)]+"updated_"+filenames[i][len(pathh):], "w") as f:
        for j in stringer(txt_maker(filenames)[i]):
            f.write(j+'\n')

In [334]:
#2. consolidating updated turnstile files


with open('C:\\Users\\PC\\Desktop\\Work\\C1\\master_turnstile.csv', 'w') as master_turnstile:
        master_turnstile.write('C/A,UNIT,SCP,DATEn,TIMEn,DESCn,ENTRIESn,EXITSn\n')
        for filename in filenames:
            with open(filename[:len(pathh)]+"updated_"+filename[len(pathh):], 'r') as f:
                s = f.read()
                master_turnstile.write(s)

In [335]:
#3. Choose those where DESCn is REGULAR
turnstile_data = pd.read_csv('C:\\Users\\PC\\Desktop\\Work\\C1\\master_turnstile.csv')
df = pd.DataFrame(turnstile_data)

In [336]:
#4.
def entries_difference(df):
    
    ENTRIESn_hourly=[]
    for i in range(len(df['EXITSn'])):
        try:
            if ( (df['C/A'][i] == df['C/A'][i-1]) & (df['UNIT'][i] == df['UNIT'][i-1]) & (df['SCP'][i] == df['SCP'][i-1]) ):    
                ENTRIESn_hourly.append(abs(df['ENTRIESn'][i]-df['ENTRIESn'][i-1]))     
            else:
                ENTRIESn_hourly.append(0)
        except:
            ENTRIESn_hourly.append(0)
    df['ENTRIESn_hourly'] = ENTRIESn_hourly 
    return df



def exits_difference(df):
    
    EXITSn_hourly=[]
    for i in range(len(df['EXITSn'])):
        try:
            if ( (df['C/A'][i] == df['C/A'][i-1]) & (df['UNIT'][i] == df['UNIT'][i-1]) & (df['SCP'][i] == df['SCP'][i-1]) ):    
                EXITSn_hourly.append(abs(df['EXITSn'][i]-df['EXITSn'][i-1]))     
            else:
                EXITSn_hourly.append(0)
        except:
            EXITSn_hourly.append(0)
    df['EXITSn_hourly'] = EXITSn_hourly 
    return df

In [337]:
df = exits_difference(df)
df = entries_difference(df)

In [338]:
#5
def time_to_hour(time):
    try:
        hours = int(str(time)[1]) if str(time)[0] == '0' else int(str(time)[0:2])
        return hours
    except:
        pass

A=list(df['TIMEn'])
for i in range(len(A)):
    A[i]=time_to_hour(A[i])
df['TIMEn'] = A 

In [339]:
#6.
def reformat_subway_dates(date):
    date=date[:5]+'-20'+date[6:]
    date_formatted = datetime.datetime.strptime(date, "%m-%d-%Y").strftime("%Y-%m-%d")
    return date_formatted

A=list(df['DATEn'])
for i in range(len(A)):
    A[i]=reformat_subway_dates(A[i])
df['DATEn'] = A 

In [340]:
df

,C/A,UNIT,SCP,DATEn,TIMEn,DESCn,ENTRIESn,EXITSn,EXITSn_hourly,ENTRIESn_hourly
0,A002,R051,02-00-00,2011-04-30,0,REGULAR,3143506,1087907,0,0
1,A002,R051,02-00-00,2011-04-30,4,REGULAR,3143547,1087915,8,41
2,A002,R051,02-00-00,2011-04-30,8,REGULAR,3143563,1087935,20,16
3,A002,R051,02-00-00,2011-04-30,12,REGULAR,3143646,1088024,89,83
4,A002,R051,02-00-00,2011-04-30,16,REGULAR,3143865,1088083,59,219
...,...,...,...,...,...,...,...,...,...,...
218441,A042,R086,01-00-04,2011-05-27,4,REGULAR,915737,4179675,28,13
218442,A042,R086,01-00-04,2011-05-27,8,REGULAR,915743,4179820,145,6
218443,A042,R086,01-00-04,2011-05-27,12,REGULAR,915806,4180746,926,63
218444,A042,R086,01-00-04,2011-05-27,16,REGULAR,916090,4181746,1000,284
